# Assignment 2: Building a Simple Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [1]:
Summaries_file = 'data/emotion_Summaries.pkl.bz2'
Abstracts_file = 'data/emotion_Abstracts.pkl.bz2'

In [2]:
import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )
    
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for an example of a paper:

In [3]:
Summaries[34644547]

paper(title='Visual and auditory brain areas share a representational structure that supports emotion perception.', authors=['Sievers B', 'Parkinson C', 'Kohler PJ', 'Hughes JM', 'Fogelson SV', 'Wheatley T'], year=2021, doi='10.1016/j.cub.2021.09.043')

In [4]:
Abstracts[34644547]

'Emotionally expressive music and dance occur together across the world. This may be because features shared across the senses are represented the same way even in different sensory brain areas, putting music and movement in directly comparable terms. These shared representations may arise from a general need to identify environmentally relevant combinations of sensory features, particularly those that communicate emotion. To test the hypothesis that visual and auditory brain areas share a representational structure, we created music and animation stimuli with crossmodally matched features expressing a range of emotions. Participants confirmed that each emotion corresponded to a set of features shared across music and movement. A subset of participants viewed both music and animation during brain scanning, revealing that representations in auditory and visual brain areas were similar to one another. This shared representation captured not only simple stimulus features but also combinat

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive. We will improve them in a later assignment.

In [5]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Emotionally expressive music and dance occur together across the world.")))





['emotionally', 'expressive', 'music', 'and', 'dance', 'occur', 'together', 'across', 'the', 'world.']


In [6]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(34644547, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the titles and abstracts of the papers in our dataset. We will implement our inverted index as a Python dictionary with term strings as keys and posting lists (implemented as Python lists) as values. We include all the tokens we can find in the title and (if available) in the abstract:

In [7]:
from collections import defaultdict

inverted_index = defaultdict(list)

# This can take a few seconds:
for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

Let's see what's in the index for the example term 'music':

In [8]:
print(inverted_index['music'])

[616164, 1858435, 2631969, 2728537, 3835976, 6153964, 7266193, 7510804, 7870518, 8259767, 8541967, 9076708, 9567714, 9602525, 9606949, 9818509, 9949709, 10053235, 10138592, 10260225, 10586571, 11129375, 11161359, 11407962, 11573015, 11815880, 11915127, 11990972, 12133576, 12296486, 12575587, 12678572, 12761468, 12795548, 12956543, 14565905, 14589533, 14681113, 15053726, 15182241, 15315187, 15327344, 15699060, 15709930, 15913388, 15993964, 16078183, 16086981, 16271503, 16384553, 16447368, 16458860, 16597758, 16597772, 16597776, 16597795, 16671835, 16761812, 16828911, 16935277, 16959817, 16970965, 17290372, 17295965, 17323817, 17343712, 17395609, 17416488, 17466401, 17516815, 17554674, 17695356, 17965981, 17997625, 18039047, 18189577, 18392933, 18501389, 18505596, 18558143, 18588701, 18710596, 18729581, 18753014, 18755225, 18824047, 18826699, 18985111, 18988943, 19102595, 19227107, 19253088, 19256729, 19368861, 19391047, 19526057, 19588251, 19592998, 19594648, 19673788, 19673808, 1967380

Wow that's a lot of research related to music and emotion! We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'dance':

In [9]:
query_word = 'dance'
for i in inverted_index[query_word]:
    display_summary(i, show_abstract=True)

----------

# Tasks

**Your name:** ...

### Task 1

Implement the function `merge_and` outlined below. This function takes two posting lists from the index that can be assumed to be sorted already (e.g. [3,5,8,10] and [5,7,8,12]), and it should return the result of the merging of the two lists with AND. The resulting list should therefore include all the elements that appear in both lists. As explained on the slides, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Then, test your function with an example.

In [10]:
# your code here



### Task 2

Similarly as above, implement the function `merge_or` outlined below that executes an OR merging of the lists. The resulting list should therefore include all the elements that appear in at least one of the lists. Again, this operation should take advantage of the input lists being sorted already, should not perform any additional sorting operation, and should go through each of the input lists just once. Elements that appear in both input list should only appear once in the output list. Test your function again with an example.

In [11]:
# your code here



### Task 3

Construct a function called `query_and` that takes as input a single string, consisting of one or more words, and returns as function value a list of matching documents. `query_and`, as its name suggests, should require that all query terms are present in the documents of the result list.

For that, access the variable `inverted_index` from above and use the method `merge_and` that you defined. Also use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query.

Again demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

In [12]:
# your code here



### Task 4

Construct another function called `query_or` that works in the same way as `query_and` you just implemented, but returns as function value the documents that contain _at least one_ of the words in the query, using the `merge_or` function you defined.

Demonstrate the working of this function also with an example (again, choose one that leads to fewer than 100 hits).

In [13]:
# your code here



### Task 5

Why does `query_and('visual complex scenes emotion')` not return our example paper 31588367, even though it mentions these terms in the title and abstract? (You do not have to implement anything to fix this yet!)

**Answer:** 

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.